## 1. Business Understanding

### Project Overview
SyriaTel, a telecommunications company, is experiencing customer churn —customers leaving the service which leads to loss of revenue and market share. The business wants to predict which customers are likely to churn, so they can intervene before it's too late.

### Objective
The goal of this project is to build a **binary classification model** that predicts whether a customer is likely to churn based on historical usage and demographic data.

### Stakeholders
- **Customer Retention Team**: Needs a list of customers likely to churn to implement retention strategies.
- **Marketing Team**: Wants to know which features drive churn so they can adjust offers and messaging.
- **Executive Team**: Needs a high-level summary of churn trends and ROI on churn-prevention efforts.

### Machine Learning Framing
We will frame this as a **supervised binary classification problem**, where the target is whether the customer has churned or not.


## 2. Data Understanding

In this step, we aim to get a clear overview of the dataset:

- Load and preview the data
- Examine feature types and basic statistics
- Identify missing values and duplicates
- Understand the target variable distribution

These insights will help guide our data preparation and modeling strategy.The dataset being used for this project was obtained from kaggle.
